# time - Clock Time

Purpose:	Functions for manipulating clock time.

The time module provides access to several different types of clocks, each useful for different purposes. The standard system calls like time() report the system “wall clock” time. The monotonic() clock can be used to measure elapsed time in a long-running process because it is guaranteed never to move backwards, even if the system time is changed. For performance testing, perf_counter() provides access to the clock with the highest available resolution to make short time measurements more accurate. The CPU time is available through clock(), and process_time() returns the combined processor time and system time.

#### Note
The implementations expose C library functions for manipulating dates and times. Since they are tied to the underlying C implementation, some details (such as the start of the epoch and maximum date value supported) are platform-specific. Refer to the library documentation for complete details.

## Comparing Clocks

Implementation details for the clocks varies by platform. Use get_clock_info() to access basic information about the current implementation, including the clock’s resolution.

This output for Mac OS X shows that the monotonic and perf_counter clocks are implemented using the same underlying system call.

In [1]:
# time_get_clock_info.py

import textwrap
import time

available_clocks = [
    ('clock', time.clock),
    ('monotonic', time.monotonic),
    ('perf_counter', time.perf_counter),
    ('process_time', time.process_time),
    ('time', time.time),
]

for clock_name, func in available_clocks:
    print(textwrap.dedent('''\
    {name}:
        adjustable     : {info.adjustable}
        implementaion  : {info.implementation}
        monotonic      : {info.monotonic}
        resolution     : {info.resolution}
        current        : {current} 
    ''').format(
        name=clock_name,
        info=time.get_clock_info(clock_name),
        current=func()
    ))



clock:
    adjustable     : False
    implementaion  : clock()
    monotonic      : True
    resolution     : 1e-06
    current        : 1.006665 

monotonic:
    adjustable     : False
    implementaion  : mach_absolute_time()
    monotonic      : True
    resolution     : 1e-09
    current        : 270199.643583514 

perf_counter:
    adjustable     : False
    implementaion  : mach_absolute_time()
    monotonic      : True
    resolution     : 1e-09
    current        : 270199.64374085 

process_time:
    adjustable     : False
    implementaion  : getrusage(RUSAGE_SELF)
    monotonic      : True
    resolution     : 1e-06
    current        : 1.0090109999999999 

time:
    adjustable     : True
    implementaion  : gettimeofday()
    monotonic      : False
    resolution     : 1e-06
    current        : 1490938451.065118 



## Wall Clock Time

One of the core functions of the time module is time(), which returns the number of seconds since the start of the “epoch” as a floating point value.

The epoch is the start of measurement for time, which for Unix systems is 0:00 on January 1, 1970. Although the value is always a float, actual precision is platform-dependent.

In [2]:
#time_time.py
import time
print('the time is: ', time.time())

the time is:  1490938451.073378


The float representation is useful when storing or comparing dates, but not as useful for producing human readable representations. For logging or printing time ctime() can be more useful.

The second print() call in this example shows how to use ctime() to format a time value other than the current time.

In [3]:
# time_ctime.py
print('the time is: ', time.ctime())

later = time.time() + 15

print ('15 sec from now: ', time.ctime(later))

the time is:  Fri Mar 31 01:34:11 2017
15 sec from now:  Fri Mar 31 01:34:26 2017


## Monotonic Clocks

Because time() looks at the system clock, and the system clock can be changed by the user or system services for synchronizing clocks across multiple computers, calling time() repeatedly may produce values that go forwards and backwards. This can result in unexpected behavior when trying to measure durations or otherwise use those times for computation. Avoid those situations by using monotonic(), which always returns values that go forward.

The start point for the monotonic clock is not defined, so return values are only useful for doing calculations with other clock values. In this example the duration of the sleep is measured using monotonic().

In [4]:
# time_monotonic.py
import time
start = time.monotonic()
time.sleep(0.1)
end = time.monotonic()
print('start : {:>9.2f}'.format(start))
print('end   : {:>9.2f}'.format(end))
print('span  : {:>9.2f}'.format(end - start))

start : 270199.67
end   : 270199.78
span  :      0.10


## Processor Clock Time

While time() returns a wall clock time, clock() returns processor clock time. The values returned from clock() reflect the actual time used by the program as it runs.

In this example, the formatted ctime() is printed along with the floating point values from time(), and clock() for each iteration through the loop.

#### Note
If you want to run the example on your system, you may have to add more cycles to the inner loop or work with a larger amount of data to actually see a difference in the times.

In [5]:
# time_clock.py

import hashlib
import time

# Data to use to calculate md5 checksums
data = open('example.txt','rb').read()

for i in range(5):
    h = hashlib.sha1()
    print(time.ctime(), ': {:0.3f} {:0.3f}'.format(time.time(), time.clock()))
    for i in range(8000000):
        h.update(data)
    cksum = h.digest()

Fri Mar 31 01:34:11 2017 : 1490938451.212 1.058
Fri Mar 31 01:34:14 2017 : 1490938454.720 4.548
Fri Mar 31 01:34:18 2017 : 1490938458.198 8.012
Fri Mar 31 01:34:21 2017 : 1490938461.665 11.470
Fri Mar 31 01:34:25 2017 : 1490938465.353 15.113


Typically, the processor clock does not tick if a program is not doing anything

In this example, the loop does very little work by going to sleep after each iteration. The time() value increases even while the application is asleep, but the clock() value does not.

In [6]:
# time_clock_sleep.py

import time 
template = '{} - {:0.2f} - {:0.2f}'

print(template.format(time.ctime(), time.time(), time.clock()))

for i in range(3,0,-1):
    print('Sleeping', i)
    time.sleep(i)
    print(template.format(time.ctime(), time.time(), time.clock()))

Fri Mar 31 01:34:29 2017 - 1490938469.53 - 19.05
Sleeping 3
Fri Mar 31 01:34:32 2017 - 1490938472.53 - 19.06
Sleeping 2
Fri Mar 31 01:34:34 2017 - 1490938474.53 - 19.06
Sleeping 1
Fri Mar 31 01:34:35 2017 - 1490938475.54 - 19.06


Calling sleep() yields control from the current thread and asks it to wait for the system to wake it back up. If a program has only one thread, this effectively blocks the app and it does no work.

## Performance Counter

It is important to have a high-resolution monotonic clock for measuring performance. Determining the best clock data source requires platform-specific knowledge, which Python provides in perf_counter().

As with monotonic(), the epoch for perf_counter() is undefined, and the values are meant to be used for comparing and computing values, not as absolute times.

In [7]:
# time_perf_counter.py

import hashlib
import time

# Data to use to calculate md5 checksums
data = open('example.txt','rb').read()

loop_start = time.perf_counter()

for i in range(5):
    iter_start = time.perf_counter()
    h = hashlib.sha1()
    for i in range(3000000):
        h.update(data)
    chksum = h.digest()
    now = time.perf_counter()
    loop_elapsed = now - loop_start
    iter_elapsed = now - iter_start
    print(time.ctime(), ': {:0.4f} {:0.4f}'.format(iter_elapsed, loop_elapsed))

Fri Mar 31 01:34:36 2017 : 1.3089 1.3091
Fri Mar 31 01:34:38 2017 : 1.3356 2.6450
Fri Mar 31 01:34:39 2017 : 1.2359 3.8813
Fri Mar 31 01:34:40 2017 : 1.2252 5.1066
Fri Mar 31 01:34:42 2017 : 1.4495 6.5564


## Time Components

Storing times as elapsed seconds is useful in some situations, but there are times when a program needs to have access to the individual fields of a date (year, month, etc.). The time module defines struct_time for holding date and time values with components broken out so they are easy to access. There are several functions that work with struct_time values instead of floats.

The gmtime() function returns the current time in UTC. localtime() returns the current time with the current time zone applied. mktime() takes a struct_time and converts it to the floating point representation.

In [8]:
# time_struct.py

import time

def show_struct(s):
    print('  tm_year :', s.tm_year)
    print('  tm_mon  :', s.tm_mon)
    print('  tm_mday :', s.tm_mday)
    print('  tm_hour :', s.tm_hour)
    print('  tm_min  :', s.tm_min)
    print('  tm_sec  :', s.tm_sec)
    print('  tm_wday :', s.tm_wday)
    print('  tm_yday :', s.tm_yday)
    print('  tm_isdst:', s.tm_isdst)
    
print('gmtime:')
show_struct(time.gmtime())
print('\nlocaltime:')
show_struct(time.localtime())
print('\nmktime     :', time.mktime(time.localtime()))
print('time.time():', time.time())

gmtime:
  tm_year : 2017
  tm_mon  : 3
  tm_mday : 31
  tm_hour : 5
  tm_min  : 34
  tm_sec  : 42
  tm_wday : 4
  tm_yday : 90
  tm_isdst: 0

localtime:
  tm_year : 2017
  tm_mon  : 3
  tm_mday : 31
  tm_hour : 1
  tm_min  : 34
  tm_sec  : 42
  tm_wday : 4
  tm_yday : 90
  tm_isdst: 1

mktime     : 1490938482.0
time.time(): 1490938482.148906


## Working with Time Zones

The functions for determining the current time depend on having the time zone set, either by the program or by using a default time zone set for the system. Changing the time zone does not change the actual time, just the way it is represented.

To change the time zone, set the environment variable TZ, then call tzset(). The time zone can be specified with a lot of detail, right down to the start and stop times for daylight savings time. It is usually easier to use the time zone name and let the underlying libraries derive the other information, though.

This example program changes the time zone to a few different values and shows how the changes affect other settings in the time module.

The default time zone on the system used to prepare the examples is US/Eastern. The other zones in the example change the tzname, daylight flag, and timezone offset value.

In [9]:
# time_timezone.py
import time
import os


def show_zone_info():
    print('  TZ    :', os.environ.get('TZ', '(not set)'))
    print('  tzname:', time.tzname)
    print('  Zone  : {} ({})'.format(
        time.timezone, (time.timezone / 3600)))
    print('  DST   :', time.daylight)
    print('  Time  :', time.ctime())
    print()

print('Default :')
show_zone_info()

ZONES = [
    'GMT',
    'Europe/Amsterdam',
]

for zone in ZONES:
    os.environ['TZ'] = zone
    time.tzset()
    print(zone, ':')
    show_zone_info()

Default :
  TZ    : (not set)
  tzname: ('EST', 'EDT')
  Zone  : 18000 (5.0)
  DST   : 1
  Time  : Fri Mar 31 01:34:42 2017

GMT :
  TZ    : GMT
  tzname: ('GMT', 'GMT')
  Zone  : 0 (0.0)
  DST   : 0
  Time  : Fri Mar 31 05:34:42 2017

Europe/Amsterdam :
  TZ    : Europe/Amsterdam
  tzname: ('CET', 'CEST')
  Zone  : -3600 (-1.0)
  DST   : 1
  Time  : Fri Mar 31 07:34:42 2017



## Parsing and Formatting Times

The two functions strptime() and strftime() convert between struct_time and string representations of time values. There is a long list of formatting instructions available to support input and output in different styles. The complete list is documented in the library documentation for the time module.

This example converts the current time from a string to a struct_time instance and back to a string.

The output string is not exactly like the input, since the day of the month is prefixed with a zero.

In [10]:
# time_strptime.py
import time


def show_struct(s):
    print('  tm_year :', s.tm_year)
    print('  tm_mon  :', s.tm_mon)
    print('  tm_mday :', s.tm_mday)
    print('  tm_hour :', s.tm_hour)
    print('  tm_min  :', s.tm_min)
    print('  tm_sec  :', s.tm_sec)
    print('  tm_wday :', s.tm_wday)
    print('  tm_yday :', s.tm_yday)
    print('  tm_isdst:', s.tm_isdst)

now = time.ctime(1483391847.433716)
print('Now:', now)

parsed = time.strptime(now)
print('\nparsed : ', parsed)
print('\nParsed:')
show_struct(parsed)

print('\nFormatted:',
      time.strftime("%a %b %d %H:%M:%S %Y", parsed))

Now: Mon Jan  2 22:17:27 2017

parsed :  time.struct_time(tm_year=2017, tm_mon=1, tm_mday=2, tm_hour=22, tm_min=17, tm_sec=27, tm_wday=0, tm_yday=2, tm_isdst=-1)

Parsed:
  tm_year : 2017
  tm_mon  : 1
  tm_mday : 2
  tm_hour : 22
  tm_min  : 17
  tm_sec  : 27
  tm_wday : 0
  tm_yday : 2
  tm_isdst: -1

Formatted: Mon Jan 02 22:17:27 2017
